In [2]:
import os
import logging

logger = logging.getLogger(__name__)

os.chdir("../")

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from src.etl import *
from src.rfm import *
from src.utils import *

In [ ]:
data